### Black Box Attacks and Defenses 

## Selecting 1000 images from ImageNet

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from torchvision import transforms
import torchvision.models as models

import matplotlib.pyplot as plt 
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
from art.estimators.classification import PyTorchClassifier

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
def softmax_activation(inputs): 
    inputs = inputs.tolist()
    exp_values = np.exp(inputs - np.max(inputs)) 
    # Normalize 
    probabilities = exp_values / np.sum(exp_values)

    return probabilities

In [5]:
IMAGE_DIR = 'data/imagenetv2/imagenetv2-top-images-format-val'

### Loading Models 

In [6]:
resnet18 = models.resnet18(pretrained=True) 
resnet34 = models.resnet34(pretrained=True) 
resnet50 = models.resnet50(pretrained=True) 
resnet101 = models.resnet101(pretrained=True)
resnet152 = models.resnet152(pretrained=True)  

vgg11 = models.vgg11(pretrained=True)
vgg13 = models.vgg13(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
vgg19 = models.vgg19(pretrained=True)

densenet121 = models.densenet161(pretrained=True)
densenet161 = models.densenet161(pretrained=True)
densenet169 = models.densenet161(pretrained=True)
densenet201 = models.densenet161(pretrained=True)

In [7]:
models_list = [resnet18, resnet34, resnet50, resnet101, resnet152, 
               vgg11, vgg13, vgg16, vgg19,
               densenet121, densenet161, densenet169, densenet201] 

In [8]:
def select_images(models_list, num_images): 
    image_path_list = [] 
    for i in range (0, 10): 
        for j in range(0, 1000): 
            try: 
                image_path = IMAGE_DIR + '/{}/{}.jpeg'.format(j, i)
                input_image = Image.open(image_path) 
                input_tensor = preprocess(input_image)
                input_batch = input_tensor.unsqueeze(0).numpy().astype(np.float32)
                criterion = nn.CrossEntropyLoss()

                selected = True 
                for model in models_list: 
                    # Create the ART classifier
                    classifier = PyTorchClassifier(
                        model=model,
                        loss=criterion,
                        input_shape=(3, 224, 224),
                        nb_classes=1000,
                        device_type='gpu'
                    )
                    prediction = classifier.predict(input_batch) 
                    predicted_label = np.argmax(prediction, axis=1)[0] 
                    accuracy = round(np.max(softmax_activation(prediction), axis=1)[0] * 100, 2)  

                    if predicted_label != j or accuracy < 65.0: 
                        selected = False 
        
                if selected: 
                    image_path_list.append('/{}/{}.jpeg'.format(j, i))
                    if len(image_path_list)%20 == 0: 
                        print ("Completed: {}/{}".format(len(image_path_list), num_images))

                    if len(image_path_list) == num_images: 
                        return image_path_list
            except: 
                print ("Error: File does not appear to exist.")

    return image_path_list

In [9]:
images_list = select_images(models_list, 1000)

Completed: 20/1000
Completed: 40/1000
Completed: 60/1000
Completed: 80/1000
Error: File does not appear to exist.
Completed: 100/1000
Completed: 120/1000
Completed: 140/1000
Completed: 160/1000
Completed: 180/1000
Completed: 200/1000
Completed: 220/1000
Completed: 240/1000
Completed: 260/1000
Completed: 280/1000
Completed: 300/1000
Completed: 320/1000
Completed: 340/1000
Completed: 360/1000
Completed: 380/1000
Completed: 400/1000
Completed: 420/1000
Completed: 440/1000
Completed: 460/1000
Completed: 480/1000
Completed: 500/1000
Completed: 520/1000
Completed: 540/1000
Completed: 560/1000
Completed: 580/1000
Completed: 600/1000
Completed: 620/1000
Completed: 640/1000
Completed: 660/1000
Completed: 680/1000
Completed: 700/1000
Completed: 720/1000
Completed: 740/1000
Completed: 760/1000
Completed: 780/1000
Completed: 800/1000
Completed: 820/1000
Completed: 840/1000
Completed: 860/1000
Completed: 880/1000
Completed: 900/1000
Completed: 920/1000
Completed: 940/1000
Completed: 960/1000
Comple

In [11]:
IMAGE_DIR = 'data'

file_path = "{}/1000_images_paths_3_filter65.txt".format(IMAGE_DIR)
with open(file_path, "w") as output:
    output.write(str(images_list))